In [1]:
# pip install scikit-learn pandas scipy
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
import os
import sys
import pandas as pd
import numpy as np
import scipy as sp
from sklearn import metrics

LR classifier: 
The gridsearch parameters initialised for the logistic regression classifier are described and rationalised below. For more on the parameters dicussed [see the scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression).

- solvers = ["lbfgs", "saga", "liblinear"]

The solver determines the algorithm used when optimizing. Different solvers are avilable, 
'lbfgs' is robust and the default solver when using ```scikit-learn``` LogisticRegression. Additionally, 'saga' and 'liblinear' are included in the tuning. 'liblinear' is recommended on smaller datasets, which the *Fake or Real News* dataset is, and 'saga' is overall well performing. 

- penalties = ["l1", "l2", "None"]

*consider whether lbfgs should be included? it does not support l1 penalty*

The penalties represent different regularization techinques, which helps balance between model fit and complexity. Not all solvers support all penalties, in this case 'l1' and 'l2' penalties have been included in the gridsearch, as well as 'None'. 

- C = [1.0, 0.1, 0.01]

The C hyperparameter defines the strength of the regulrazation on the model, the smaller the value the more regulration and the simpler the model, in turn a higher value slacks the regularization and allows for a more complex model. The default is 1.0, and 0.1 and 0.01 has also been included as paramaters in the grid search. 

In [2]:
y_train, y_test, X_train, X_test, feature_names = pd.read_pickle('out/features.pkl')

In [22]:
pipe = Pipeline([('classifier' , LogisticRegression(max_iter=1000))])
penalties = ['l1', 'l2', None] 
C = [1.0, 0.1, 0.01]  
solvers = ['liblinear', "saga", "lbfgs"]  
parameters = dict(classifier__penalty = penalties,  
                  classifier__C = C,
                  classifier__solver = solvers)
scores = ['precision', 'recall', 'f1']

In [23]:
for score in scores:
    print(f"# Tuning hyper-parameters for {score}")
    print()
    LRC = GridSearchCV(pipe,                                
                       parameters,                          
                       scoring= f"{score}_weighted",
                       cv=10) 
    LRC.fit(X_train, y_train)
    

    print("Best parameters set found on training data:")
    print()
    print(LRC.best_params_)
    print()
    print("Grid scores on training data:")
    print()
    means = LRC.cv_results_['mean_test_score']
    stds = LRC.cv_results_['std_test_score']
    params = LRC.cv_results_['params']

    i = 0
    for mean, stdev, param in zip(means, stds, params):
        print(f"Run {i}: {round(mean,3)} (SD=±{round(stdev*2, 3)}), using {param}")
        i += 1
    print()
    
    print("Detailed classification report:")
    print()
    print("The model is trained on the full training set.")
    print("The scores are computed on the full test set.")
    print()
    y_true, y_pred = y_test, LRC.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision



/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:35

Best parameters set found on training data:

{'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Grid scores on training data:

Run 0: 0.891 (SD=±0.024), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}
Run 1: 0.891 (SD=±0.024), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
Run 2: nan (SD=±nan), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'lbfgs'}
Run 3: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}
Run 4: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
Run 5: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}
Run 6: nan (SD=±nan), using {'classifier__C': 1.0, 'classifier__penalty': None, 'classifier__solver': 'liblinear'}
Run 7: 0.872 (SD=±0.034), using {'

/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:35

Best parameters set found on training data:

{'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Grid scores on training data:

Run 0: 0.891 (SD=±0.024), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}
Run 1: 0.891 (SD=±0.024), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
Run 2: nan (SD=±nan), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'lbfgs'}
Run 3: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}
Run 4: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
Run 5: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}
Run 6: nan (SD=±nan), using {'classifier__C': 1.0, 'classifier__penalty': None, 'classifier__solver': 'liblinear'}
Run 7: 0.872 (SD=±0.033), using {'

/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:35

Best parameters set found on training data:

{'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Grid scores on training data:

Run 0: 0.891 (SD=±0.024), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}
Run 1: 0.891 (SD=±0.024), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
Run 2: nan (SD=±nan), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'lbfgs'}
Run 3: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}
Run 4: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
Run 5: 0.885 (SD=±0.034), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}
Run 6: nan (SD=±nan), using {'classifier__C': 1.0, 'classifier__penalty': None, 'classifier__solver': 'liblinear'}
Run 7: 0.872 (SD=±0.033), using {'